## Предсказание стоимости квартиры на основе геопараметров

Проверка влияния параметров городской инфраструктуры на стоимость квартир. Насколько велика эта добавочная стоимость.

Задание и данные указаны [здесь](https://ods.ai/competitions/geo_course_real_estate).

In [1]:
!pip -q install folium matplotlib mapclassify osmnx networkx

In [ ]:
pip install osmnx

In [ ]:
pip install --upgrade pip

In [ ]:
pip install catboost

In [392]:
import pandas as pd
import numpy as np
import geopandas as gpd
import osmnx as ox
import networkx as nx

from shapely import wkt

from sklearn import metrics

from sklearn.model_selection import train_test_split
from sklearn.metrics import root_mean_squared_error
# Если колаб выдаёт ошибку при импорте метрики, перезагрузите среду выполнения
from catboost.utils import eval_metric
from catboost import CatBoostRegressor, Pool

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

Задача заключается в предсказании стоимость квартир для тестового набора данных (*test.csv*) на основе геопризнаков. Такими параметрами были выбраны пешеходная доступность от квартиры до трамвайных остановок и станций метро.

Эти виды транспорта были выбраны в виду своих преимуществ перед другими видами городского общественного транспорта (вместимость, скорость, экологичность, отсутствие пробок).

In [6]:
# Считываем данные
df_train = pd.read_csv("df_train.csv", index_col=[0])
df_test = pd.read_csv("df_test.csv", index_col=[0])

In [7]:
df_train.head()

,latitude,longitude,city,area,area_kitchen,rooms_num,has_balcony,price
0,53.2117,50.1669,Самара,33.9,8.0,1,True,2473000
1,45.0389,39.0985,Краснодар,49.8,12.0,2,True,3733000
2,45.0694,38.9716,Краснодар,45.1,25.1,2,True,3445000
3,53.2164,50.2605,Самара,70.1,10.0,3,True,4458000
4,53.1651,50.3842,Самара,34.0,NaN,1,True,1295000


In [69]:
# Создаём колонку геометрий
df_train["geometry"] = gpd.points_from_xy(
    x=df_train["longitude"], y=df_train["latitude"], crs="EPSG:4326"
)
df_test["geometry"] = gpd.points_from_xy(
    x=df_test["longitude"], y=df_test["latitude"], crs="EPSG:4326"
)

In [70]:
df_train

,latitude,longitude,city,area,area_kitchen,rooms_num,has_balcony,price,geometry
0,53.2117,50.1669,Самара,33.9,8.0,1,True,2473000,POINT (50.16690 53.21170)
1,45.0389,39.0985,Краснодар,49.8,12.0,2,True,3733000,POINT (39.09850 45.03890)
2,45.0694,38.9716,Краснодар,45.1,25.1,2,True,3445000,POINT (38.97160 45.06940)
3,53.2164,50.2605,Самара,70.1,10.0,3,True,4458000,POINT (50.26050 53.21640)
4,53.1651,50.3842,Самара,34.0,NaN,1,True,1295000,POINT (50.38420 53.16510)
...,...,...,...,...,...,...,...,...,...
8037,53.2399,50.2785,Самара,43.0,5.0,2,True,4029000,POINT (50.27850 53.23990)
8038,45.0739,38.9666,Краснодар,44.9,11.8,1,True,5263000,POINT (38.96660 45.07390)
8039,45.1127,39.0288,Краснодар,43.1,12.0,1,False,2017000,POINT (39.02880 45.11270)
8040,45.0513,39.0179,Краснодар,41.1,14.2,1,True,5617000,POINT (39.01790 45.05130)


Создаём переменные с пешеходными графами городов

In [8]:
city_1 = ox.geocode_to_gdf(["Россия, Краснодар"])
polygon_1 = city_1.iloc[0]["geometry"]
G_krasnodar = ox.graph_from_polygon(polygon_1, network_type="walk", simplify=True)
G_krasnodar = ox.project_graph(G_krasnodar)
nodes_1, edges_1 = ox.graph_to_gdfs(G_krasnodar)

In [9]:
city_2 = ox.geocode_to_gdf(["Россия, город Нижний Новгород"])
polygon_2 = city_2.iloc[0]["geometry"]
G_nizhny_novgorod = ox.graph_from_polygon(polygon_2, network_type="walk", simplify=True)
G_nizhny_novgorod = ox.project_graph(G_nizhny_novgorod)
nodes_2, edges_2 = ox.graph_to_gdfs(G_nizhny_novgorod)

In [10]:
city_3 = ox.geocode_to_gdf(["Россия, Самара"])
polygon_3 = city_3.iloc[0]["geometry"]
G_samara = ox.graph_from_polygon(polygon_3, network_type="walk", simplify=True)
G_samara = ox.project_graph(G_samara)
nodes_3, edges_3 = ox.graph_to_gdfs(G_samara)

In [11]:
# Проверим какая система координат сейчас
nodes_1.crs

<Projected CRS: EPSG:32637>
Name: WGS 84 / UTM zone 37N
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Between 36°E and 42°E, northern hemisphere between equator and 84°N, onshore and offshore. Djibouti. Egypt. Eritrea. Ethiopia. Georgia. Iraq. Jordan. Kenya. Lebanon. Russian Federation. Saudi Arabia. Somalia. Sudan. Syria. Türkiye (Turkey). Ukraine.
- bounds: (36.0, 0.0, 42.0, 84.0)
Coordinate Operation:
- name: UTM zone 37N
- method: Transverse Mercator
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [12]:
edges_1.crs

<Projected CRS: EPSG:32637>
Name: WGS 84 / UTM zone 37N
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Between 36°E and 42°E, northern hemisphere between equator and 84°N, onshore and offshore. Djibouti. Egypt. Eritrea. Ethiopia. Georgia. Iraq. Jordan. Kenya. Lebanon. Russian Federation. Saudi Arabia. Somalia. Sudan. Syria. Türkiye (Turkey). Ukraine.
- bounds: (36.0, 0.0, 42.0, 84.0)
Coordinate Operation:
- name: UTM zone 37N
- method: Transverse Mercator
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [13]:
# Выгружаем остановки общественного транспорта в городах из OpenStreetMap
tags = {"public_transport": True}

features_krasnodar = ox.features.features_from_place("Krasnodar", tags=tags)
features_krasnodar = features_krasnodar.dropna(subset=["name"]) # дополнительно удаляем остановки без названий

In [ ]:
features_krasnodar.head()

In [ ]:
# Оставляем только трамвайные остановки и станции метро (в случае с Краснодаром без остановок метро)
stops_krasnodar = features_krasnodar[features_krasnodar.tram.notnull()]
stops_krasnodar.info()

In [ ]:
# С помощью модуля representative_point уточняем точки внутри полигонов или линий, которыми могут быть
# обозначены остановки
stops_krasnodar["geometry"] = stops_krasnodar["geometry"].representative_point()
stops_krasnodar = stops_krasnodar[["name", "geometry"]]

stops_krasnodar.info()

In [26]:
# Переводим геометрию точек в корректную проекцию города, определённую ранее
stops_krasnodar_crs = (
    stops_krasnodar
    .to_crs(edges_1.crs)
)

In [27]:
# Добавляем столбец "nearest_node", куда будут добавляться ближайшие узлы графа OpenStreetMap
# к координатам, чтобы в дальнейшем по ним можно было найти кратчайший маршрут
stops_krasnodar_crs["nearest_node"] = ""

In [ ]:
stops_krasnodar_crs

In [29]:
# Устанавливаем столбец "name" в качестве индекса датафрейма, чтобы к датасету можно было обращаться по меткам
stops_krasnodar_crs = stops_krasnodar_crs.set_index("name")

In [30]:
# С помощью библиотеки osmnx ищем ближайшие узлы графа OpenStreetMap для координат
# остановочных пунктов в городе
stops_krasnodar_crs["nearest_node"] = stops_krasnodar_crs["geometry"].apply(
    lambda x: ox.distance.nearest_nodes(G_krasnodar, x.x, x.y)
)

In [31]:
stops_krasnodar_crs

,geometry,nearest_node
name,,
Западное депо,POINT (493995.116 4990515.645),1992135309
Железнодорожный вокзал,POINT (499139.634 4985157.345),484799395
улица Тульская,POINT (494456.271 4987148.925),1042862412
Индустриальная улица,POINT (497057.716 4982953.622),1999275090
улица Минская,POINT (494817.846 4987356.667),909333085
...,...,...
Гомельская улица,POINT (500121.254 4992263.498),9630264591
Ахтарская улица,POINT (500150.683 4992803.314),1959404003
Ахтарская улица,POINT (500145.731 4992898.750),1999164662


In [ ]:
stops_krasnodar_crs.info()

In [33]:
# Сохраняем датафрейм с остановками в Краснодаре в csv-файл
stops_krasnodar_crs.to_csv("stops_krasnodar_crs.csv")

In [ ]:
# Выгружаем остановки общественного транспорта в городах из OpenStreetMap
tags = {"public_transport": True}

features_nizhny = ox.features.features_from_place("Nizhny Novgorod", tags=tags)
features_nizhny = features_nizhny.dropna(subset=["name"])

features_nizhny.info()

In [ ]:
# Оставляем только трамвайные остановки и станции метро
# С помощью конкатенации собираем их вместе в новый датафрейм
stops_nizhny = pd.concat([
    features_nizhny[features_nizhny.tram.notnull()],
    features_nizhny[features_nizhny.station.notnull()]
])

stops_nizhny.info()

In [ ]:
# С помощью модуля representative_point уточняем точки внутри полигонов или линий, которыми могут быть
# обозначены остановки
stops_nizhny["geometry"] = stops_nizhny["geometry"].representative_point()
stops_nizhny = stops_nizhny[["name", "geometry"]]

stops_nizhny.info()

In [41]:
# Переводим геометрию точек в корректную проекцию города, определённую ранее
stops_nizhny_crs = (
    stops_nizhny
    .to_crs(edges_2.crs)
)

In [42]:
stops_nizhny_crs.head()

name                        geometry
element_type osmid                                                           
node         324688711   Красносельская улица  POINT (437059.380 6240885.304)
             359142331              Универмаг  POINT (429650.304 6233648.133)
             359143737  Троллейбусное депо №3  POINT (432360.789 6235826.898)
             359143826         Игарская улица  POINT (431682.606 6237896.841)
             479501580      улица Добролюбова  POINT (437777.901 6242650.618)

In [43]:
# Добавляем столбец "nearest_node", куда будут добавляться ближайшие узлы графа OpenStreetMap
# к координатам, чтобы в дальнейшем по ним можно было найти кратчайший маршрут
stops_nizhny_crs["nearest_node"] = ""

In [44]:
# Устанавливаем столбец "name" в качестве индекса датафрейма, чтобы к датасету можно было обращаться по меткам
stops_nizhny_crs = stops_nizhny_crs.set_index("name")

In [46]:
# С помощью библиотеки osmnx ищем ближайшие узлы графа OpenStreetMap для координат
# остановочных пунктов в городе
stops_nizhny_crs["nearest_node"] = stops_nizhny_crs["geometry"].apply(
    lambda x: ox.distance.nearest_nodes(G_nizhny_novgorod, x.x, x.y)
)

In [ ]:
stops_nizhny_crs.info()

In [48]:
# Сохраняем датафрейм с остановками в Нижнем Новгороде в csv-файл
stops_nizhny_crs.to_csv("stops_nizhny_crs.csv")

In [ ]:
# Выгружаем остановки общественного транспорта в городах из OpenStreetMap
tags = {"public_transport": True}

features_samara = ox.features.features_from_place("Samara", tags=tags)
features_samara = features_samara.dropna(subset=["name"])

features_samara.info()

In [ ]:
# Оставляем только трамвайные остановки и станции метро
# С помощью конкатенации собираем их вместе в новый датафрейм
stops_samara = pd.concat([
    features_samara[features_samara.tram.notnull()],
    features_samara[features_samara.station.notnull()]
])

stops_samara.info()

In [51]:
# В датасете Самары оказалось только 8 станций местного метрополитена из 10 ныне действующих,
# поэтому добавим оставшиеся 2 станции вручную
df_stops_lost = pd.DataFrame(
    {
        "name": ["Юнгородок", "Кировская"],
        "Latitude": [50.2825, 50.26963],
        "Longitude": [53.2125, 53.211348],
        "station": ["yes", "yes"]
    }
)

In [52]:
# Из датафрейма делаем геодатафрейм
gdf_stops_lost = gpd.GeoDataFrame(
    df_stops_lost, geometry=gpd.points_from_xy(df_stops_lost.Latitude, df_stops_lost.Longitude), crs="EPSG:4326"
)

In [53]:
gdf_stops_lost.head()

,name,Latitude,Longitude,station,geometry
0,Юнгородок,50.28250,53.212500,yes,POINT (50.28250 53.21250)
1,Кировская,50.26963,53.211348,yes,POINT (50.26963 53.21135)


In [ ]:
# Добавляем 2 недостающие станции метро в общий датафрейм с трамвайными остановками и станциями метро в Самаре
stops_samara = pd.concat([stops_samara, gdf_stops_lost], ignore_index= True)

stops_samara.info()

In [55]:
# Проверяем станцим самарского метрополитена и смотрим их количество
stops_samara[stops_samara.station.notnull()]

,highway,ref,geometry,bus,name,public_transport,trolleybus,railway,tram,merkator_shift:bing,...,passenger_information_display,building:colour,natural,type,departures_board,addr:housenumber,addr:street,ways,Latitude,Longitude
302,NaN,NaN,POINT (50.24844 53.21284),NaN,Безымянка,platform,NaN,station,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
303,NaN,NaN,POINT (50.23577 53.20698),NaN,Победа,platform,NaN,station,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
304,NaN,NaN,POINT (50.22044 53.20155),NaN,Советская,platform,NaN,station,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
305,NaN,NaN,POINT (50.19958 53.20096),NaN,Спортивная,platform,NaN,station,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
306,NaN,NaN,POINT (50.17651 53.20027),NaN,Гагаринская,platform,NaN,station,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
307,NaN,NaN,POINT (50.16050 53.20281),NaN,Московская,platform,NaN,station,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
308,NaN,NaN,POINT (50.14899 53.21192),NaN,Российская,platform,NaN,station,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
309,NaN,NaN,POINT (50.13420 53.20958),NaN,Алабинская,platform,NaN,station,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
310,NaN,NaN,POINT (50.28250 53.21250),NaN,Юнгородок,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50.28250,53.212500
311,NaN,NaN,POINT (50.26963 53.21135),NaN,Кировская,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50.26963,53.211348


In [56]:
# С помощью модуля representative_point уточняем точки внутри полигонов или линий, которыми могут быть
# обозначены остановки
stops_samara["geometry"] = stops_samara["geometry"].representative_point()
stops_samara = stops_samara[["name", "geometry"]]

stops_samara

,name,geometry
0,Храм Всех Святых,POINT (50.15169 53.19461)
1,улица Пензенская,POINT (50.13767 53.18914)
2,Железнодорожный вокзал,POINT (50.12287 53.18811)
3,парк Победы,POINT (50.20624 53.19209)
4,Автостанция «Аврора»,POINT (50.18850 53.19158)
...,...,...
307,Московская,POINT (50.16050 53.20281)
308,Российская,POINT (50.14899 53.21192)
309,Алабинская,POINT (50.13420 53.20958)
310,Юнгородок,POINT (50.28250 53.21250)


In [57]:
# Переводим геометрию точек в корректную проекцию города, определённую ранее
stops_samara_crs = (
    stops_samara
    .to_crs(edges_3.crs)
)

In [58]:
# Добавляем столбец "nearest_node", куда будут добавляться ближайшие узлы графа OpenStreetMap
# к координатам, чтобы в дальнейшем по ним можно было найти кратчайший маршрут
stops_samara_crs["nearest_node"] = ""

In [59]:
# Устанавливаем столбец "name" в качестве индекса датафрейма, чтобы к датасету можно было обращаться по меткам
stops_samara_crs = stops_samara_crs.set_index("name")

In [62]:
# С помощью библиотеки osmnx ищем ближайшие узлы графа OpenStreetMap для координат
# остановочных пунктов в городе
stops_samara_crs["nearest_node"] = stops_samara_crs["geometry"].apply(
    lambda x: ox.distance.nearest_nodes(G_samara, x.x, x.y)
)

In [64]:
# Сохраняем датафрейм с остановками в Самаре в csv-файл
stops_samara_crs.to_csv("stops_samara_crs.csv")

In [71]:
# Копируем тренировочный датасет с квартирами в отдельный датафрейм, чтобы в нём собрать ближайшие узлы
# графа OpenStreetMap для координат квартир в городах
df_train_nodes = df_train.copy()

In [ ]:
# Для этого добавляем столбец "geometry_crs"
df_train_nodes["geometry_crs"] = ""

df_train_nodes

In [ ]:
# Переводим координаты квартир в корректные для городов проекции, определённые ранее 
count_ = 0 # задаём счётчик для проверки количества проработанных объектов

for i in df_train_nodes.index:
    flat_crs = gpd.GeoSeries(df_train_nodes.at[i, "geometry"], crs="EPSG:4326")
    if df_train_nodes.at[i, "city"] == "Краснодар":
        df_train_nodes.at[i, "geometry_crs"] = flat_crs.to_crs(edges_1.crs)[0]
    elif df_train_nodes.at[i, "city"] == "Нижний Новгород":
        df_train_nodes.at[i, "geometry_crs"] = flat_crs.to_crs(edges_2.crs)[0]
    elif df_train_nodes.at[i, "city"] == "Самара":
        df_train_nodes.at[i, "geometry_crs"] = flat_crs.to_crs(edges_3.crs)[0]
        
    count_ += 1
    print(count_)

In [74]:
df_train_nodes

,latitude,longitude,city,area,area_kitchen,rooms_num,has_balcony,price,geometry,geometry_crs
0,53.2117,50.1669,Самара,33.9,8.0,1,True,2473000,POINT (50.16690 53.21170),POINT (444364.7583481302 5896144.765920336)
1,45.0389,39.0985,Краснодар,49.8,12.0,2,True,3733000,POINT (39.09850 45.03890),POINT (507758.0518276638 4987276.430569851)
2,45.0694,38.9716,Краснодар,45.1,25.1,2,True,3445000,POINT (38.97160 45.06940),POINT (497764.34935612 4990660.299869019)
3,53.2164,50.2605,Самара,70.1,10.0,3,True,4458000,POINT (50.26050 53.21640),POINT (450620.75961276994 5896598.90707207)
4,53.1651,50.3842,Самара,34.0,NaN,1,True,1295000,POINT (50.38420 53.16510),POINT (458831.4735939478 5890813.869220274)
...,...,...,...,...,...,...,...,...,...,...
8037,53.2399,50.2785,Самара,43.0,5.0,2,True,4029000,POINT (50.27850 53.23990),POINT (451849.04119317944 5899200.85693531)
8038,45.0739,38.9666,Краснодар,44.9,11.8,1,True,5263000,POINT (38.96660 45.07390),POINT (497370.9552071012 4991160.3494087905)
8039,45.1127,39.0288,Краснодар,43.1,12.0,1,False,2017000,POINT (39.02880 45.11270),POINT (502265.426288711 4995470.469169812)
8040,45.0513,39.0179,Краснодар,41.1,14.2,1,True,5617000,POINT (39.01790 45.05130),POINT (501409.53431823035 4988649.361517621)


In [ ]:
df_train_nodes["nearest_node"] = ""

df_train_nodes

In [ ]:
# С помощью библиотеки osmnx ищем ближайшие узлы графа OpenStreetMap для координат
# квартир в каждом городе
count_ = 0 # задаём счётчик для проверки количества проработанных объектов

for i in df_train_nodes.index:
    if df_train_nodes.at[i, "city"] == "Краснодар":
        df_train_nodes.at[i, "nearest_node"] = ox.distance.nearest_nodes(G_krasnodar, df_train_nodes.at[i, "geometry_crs"].x, df_train_nodes.at[i, "geometry_crs"].y)
        print(f'1 - {df_train_nodes.at[i, "geometry_crs"]}')
        print(f'2 - {df_train_nodes.at[i, "nearest_node"]}')
    elif df_train_nodes.at[i, "city"] == "Нижний Новгород":
        df_train_nodes.at[i, "nearest_node"] = ox.distance.nearest_nodes(G_nizhny_novgorod, df_train_nodes.at[i, "geometry_crs"].x, df_train_nodes.at[i, "geometry_crs"].y)
        print(f'1 - {df_train_nodes.at[i, "geometry_crs"]}')
        print(f'2 - {df_train_nodes.at[i, "nearest_node"]}')
    elif df_train_nodes.at[i, "city"] == "Самара":
        df_train_nodes.at[i, "nearest_node"] = ox.distance.nearest_nodes(G_samara, df_train_nodes.at[i, "geometry_crs"].x, df_train_nodes.at[i, "geometry_crs"].y)
        print(f'1 - {df_train_nodes.at[i, "geometry_crs"]}')
        print(f'2 - {df_train_nodes.at[i, "nearest_node"]}')
        
    count_ += 1
    print(count_)

In [80]:
df_train_nodes

,latitude,longitude,city,area,area_kitchen,rooms_num,has_balcony,price,geometry,geometry_crs,nearest_node
0,53.2117,50.1669,Самара,33.9,8.0,1,True,2473000,POINT (50.16690 53.21170),POINT (444364.7583481302 5896144.765920336),1376917912
1,45.0389,39.0985,Краснодар,49.8,12.0,2,True,3733000,POINT (39.09850 45.03890),POINT (507758.0518276638 4987276.430569851),533897701
2,45.0694,38.9716,Краснодар,45.1,25.1,2,True,3445000,POINT (38.97160 45.06940),POINT (497764.34935612 4990660.299869019),921918400
3,53.2164,50.2605,Самара,70.1,10.0,3,True,4458000,POINT (50.26050 53.21640),POINT (450620.75961276994 5896598.90707207),10828204246
4,53.1651,50.3842,Самара,34.0,NaN,1,True,1295000,POINT (50.38420 53.16510),POINT (458831.4735939478 5890813.869220274),597712746
...,...,...,...,...,...,...,...,...,...,...,...
8037,53.2399,50.2785,Самара,43.0,5.0,2,True,4029000,POINT (50.27850 53.23990),POINT (451849.04119317944 5899200.85693531),1567984170
8038,45.0739,38.9666,Краснодар,44.9,11.8,1,True,5263000,POINT (38.96660 45.07390),POINT (497370.9552071012 4991160.3494087905),11462814570
8039,45.1127,39.0288,Краснодар,43.1,12.0,1,False,2017000,POINT (39.02880 45.11270),POINT (502265.426288711 4995470.469169812),7310019142
8040,45.0513,39.0179,Краснодар,41.1,14.2,1,True,5617000,POINT (39.01790 45.05130),POINT (501409.53431823035 4988649.361517621),295196089


In [82]:
# Сохраняем датафрейм с узлами графа OpenStreetMap в csv-файл
df_train_nodes.to_csv("df_train_nodes.csv")

In [83]:
# Копируем полученный датасет с узлами графа OpenStreetMap, чтобы по этим узлам найти количество
# ближайших остановок, находящихся в пешей доступности от квартиры
df_train_nodes_copy = df_train_nodes.copy()

In [ ]:
df_train_nodes_copy["count_stops"] = ""
df_train_nodes_copy

In [ ]:
# С помощью библиотеки networkx находим остановки, находящиеся от квартиры на расстоянии не более,
# чем в 1 км (1000 м).
# Для этого пройдёмся по датасетам (с остановками в городах и с квартирами)
count_ = 0 # задаём счётчик для проверки количества проработанных объектов

for i in df_train_nodes_copy.index:
    if df_train_nodes_copy.at[i, "count_stops"] == "":
        if df_train_nodes_copy.at[i, "city"] == "Краснодар":
            count_stop = 0 # задаём счётчик для увеличения количества найденных остановок на указанном расстоянии
            for j in stops_krasnodar_crs.index:
                try:
                    '''
                    с помощью библиотеки networkx определяем расстояние между квартирой и городскими остановками
                    по графу OpenStreetMap и сравниваем полученное расстояние с допустимым
                    '''
                    route_length = nx.shortest_path_length(G_krasnodar, df_train_nodes_copy.at[i, "nearest_node"], stops_krasnodar_crs.at[j, "nearest_node"], weight="length")
                    if route_length <= 1000:
                        '''
                        в случае нахождения остановки на допустимом расстоянии
                        увеличиваем количество остановок в счётчике для итерируемой квартиры
                        '''
                        count_stop += 1
                except ValueError:
                    continue
            # обновляем значение количества остановок для итерируемой квартиры
            df_train_nodes_copy.at[i, "count_stops"] = count_stop
            print(count_)

        elif df_train_nodes_copy.at[i, "city"] == "Нижний Новгород":
            count_stop = 0 # задаём счётчик для увеличения количества найденных остановок на указанном расстоянии
            for j in stops_nizhny_crs.index:
                try:
                    '''
                    с помощью библиотеки networkx определяем расстояние между квартирой и городскими остановками
                    по графу OpenStreetMap и сравниваем полученное расстояние с допустимым
                    '''
                    route_length = nx.shortest_path_length(G_nizhny_novgorod, df_train_nodes_copy.at[i, "nearest_node"], stops_nizhny_crs.at[j, "nearest_node"], weight="length")
                    if route_length <= 1000:
                        '''
                        в случае нахождения остановки на допустимом расстоянии
                        увеличиваем количество остановок в счётчике для итерируемой квартиры
                        '''
                        count_stop += 1
                except ValueError:
                    continue
            # обновляем значение количества остановок для итерируемой квартиры
            df_train_nodes_copy.at[i, "count_stops"] = count_stop
            print(count_)

        elif df_train_nodes_copy.at[i, "city"] == "Самара":
            count_stop = 0 # задаём счётчик для увеличения количества найденных остановок на указанном расстоянии
            for j in stops_samara_crs.index:
                try:
                    '''
                    с помощью библиотеки networkx определяем расстояние между квартирой и городскими остановками
                    по графу OpenStreetMap и сравниваем полученное расстояние с допустимым
                    '''
                    route_length = nx.shortest_path_length(G_samara, df_train_nodes_copy.at[i, "nearest_node"], stops_samara_crs.at[j, "nearest_node"], weight="length")
                    if route_length <= 1000:
                        '''
                        в случае нахождения остановки на допустимом расстоянии
                        увеличиваем количество остановок в счётчике для итерируемой квартиры
                        '''
                        count_stop += 1
                except ValueError:
                    continue
            # обновляем значение количества остановок для итерируемой квартиры
            df_train_nodes_copy.at[i, "count_stops"] = count_stop
        count_ += 1
        print(f'Проработано: {count_} объектов')
    else:
        continue


In [96]:
# Сохраняем датафрейм с количеством найденных остановок в csv-файл
df_train_nodes_copy.to_csv("df_train_stops.csv")

In [97]:
# Выгружаем зелёные зоны (парки, сады, стадионы) города из OpenStreetMap
tags = {"leisure":["park", "garden", "stadium"]}

features_park_krasnodar = ox.features.features_from_place("Krasnodar", tags=tags)

In [98]:
features_park_krasnodar

barrier opening_hours  \
element_type osmid                              
node         7124254922     NaN           NaN   
way          26939146       NaN           NaN   
             26940686       NaN           NaN   
             26940707       NaN           NaN   
             26940896       NaN           NaN   
...                         ...           ...   
relation     11946152     fence           NaN   
             13059630       NaN           NaN   
             13221332       NaN           NaN   
             13264624       NaN           NaN   
             17247474       NaN           NaN   

                                                                  geometry  \
element_type osmid                                                           
node         7124254922                          POINT (39.06139 45.00442)   
way          26939146    POLYGON ((38.92787 45.05768, 38.93285 45.05487...   
             26940686    POLYGON ((38.97219 45.02357, 38.97259 45.02483...   
             26940707    POLYGON ((39.05436 45.01132, 39.05427 45.01125...   
             26940896    POLYGON ((38.96695 45.02127, 38.96744 45.02257...   
...                                                                    ...   
relation     11946152    POLYGON ((38.99728 45.02607, 38.99783 45.02621...   
             13059630    POLYGON ((38.96187 45.13623, 38.96301 45.13624...   
             13221332    POLYGON ((38.95141 45.01164, 38.95178 45.01002...   
             13264624    POLYGON ((38.97459 45.06987, 38.97458 45.06985...   
             17247474    POLYGON ((39.03493 45.04448, 39.03493 45.04489...   

                                                                   name  \
element_type osmid                                                        
node         7124254922                                          Лотосы   
way          26939146    Ботанический сад имени профессора И.С. Косенко   
             26940686                  Сквер имени маршала Г. К. Жукова   
             26940707                                  Солнечный остров   
             26940896                              Сквер Дружбы народов   
...                                                                 ...   
relation     11946152                                  Стадион "Кубань"   
             13059630                                         Санторини   
             13221332      Парк культуры и отдыха имени 30-летия Победы   
             13264624                                               NaN   
             17247474                                      Японский сад   

                        access  leisure name:ja  \
element_type osmid                                
node         7124254922    yes   garden     NaN   
way          26939146      NaN     park     NaN   
             26940686      NaN     park     NaN   
             26940707      NaN     park     NaN   
             26940896      NaN     park     NaN   
...                        ...      ...     ...   
relation     11946152      NaN  stadium     NaN   
             13059630      NaN     park     NaN   
             13221332      NaN     park     NaN   
             13264624      NaN  stadium     NaN   
             17247474      NaN     park    和風庭園   

                                                                     nodes  \
element_type osmid                                                           
node         7124254922                                                NaN   
way          26939146    [295188162, 295188161, 510916925, 2501132190, ...   
             26940686    [1809807393, 1809807429, 1809807431, 180980739...   
             26940707    [304774617, 295165235, 295165234, 1804793792, ...   
             26940896    [295169614, 295169613, 295169616, 295169615, 2...   
...                                                                    ...   
relation     11946152    [[[496246767, 521525962, 6900368698, 690036860...   
             13059630    [[[7

In [99]:
import folium
import matplotlib.pyplot as plt

In [105]:
type(features_park_krasnodar)

geopandas.geodataframe.GeoDataFrame

In [161]:
# Переводим координаты в геодатафрейме с зелёными зонами города в проекцию EPSG:32637 (UTM Zone 37N)
park_krasnodar_1500 = features_park_krasnodar.to_crs(32637)
# создаём буферную зону в размере 1500 м вокруг каждого геометрического объекта в наборе
park_krasnodar_1500["geometry"] = park_krasnodar_1500["geometry"].buffer(1500)
# переводим полученные координаты в EPSG:4326 (WGS84)
park_krasnodar_1500 = park_krasnodar_1500.to_crs(4326)

In [ ]:
park_krasnodar_1500

In [203]:
# Для дальнейшей работы создаём новый геодатафрейм на основе полученного набора данных
gdf_park_krasnodar_1500 = park_krasnodar_1500_geom.to_frame()

In [204]:
type(gdf_park_krasnodar_1500)

geopandas.geodataframe.GeoDataFrame

In [218]:
gdf_park_krasnodar_1500

geometry
element_type osmid                                                        
node         7124254922  POLYGON ((39.08043 45.00441, 39.08033 45.00308...
way          26939146    POLYGON ((38.91303 45.06615, 38.91427 45.06713...
             26940686    POLYGON ((38.96758 45.01047, 38.96579 45.01079...
             26940707    POLYGON ((39.04197 44.99280, 39.04091 44.99342...
             26940896    POLYGON ((38.96255 45.00813, 38.96063 45.00845...
...                                                                    ...
relation     11946152    POLYGON ((38.97824 45.02599, 38.97832 45.02728...
             13059630    POLYGON ((38.94279 45.13626, 38.94288 45.13757...
             13221332    POLYGON ((38.93327 45.02587, 38.93328 45.02588...
             13264624    POLYGON ((38.99357 45.06871, 38.99336 45.06756...
             17247474    POLYGON ((39.01589 45.04433, 39.01588 45.04471...

[249 rows x 1 columns]

In [227]:
# Выгружаем зелёные зоны (парки, сады, стадионы) города из OpenStreetMap
tags = {"leisure":["park", "garden", "stadium"]}

features_park_nizhny = ox.features.features_from_place("Nizhny Novgorod", tags=tags)

In [ ]:
features_park_nizhny

In [229]:
type(features_park_nizhny)

geopandas.geodataframe.GeoDataFrame

In [231]:
# Переводим координаты в геодатафрейме с зелёными зонами города в проекцию EPSG:32637 (UTM Zone 37N)
park_nizhny_1500 = features_park_nizhny.to_crs(32637)
# создаём буферную зону в размере 1500 м вокруг каждого геометрического объекта в наборе
park_nizhny_1500["geometry"] = park_nizhny_1500["geometry"].buffer(1500)
# переводим полученные координаты в EPSG:4326 (WGS84)
park_nizhny_1500 = park_nizhny_1500.to_crs(4326)

In [ ]:
park_nizhny_1500

In [234]:
# Для дальнейшей работы создаём новый геодатафрейм на основе полученного набора данных
gdf_park_nizhny_1500 = park_nizhny_1500["geometry"].to_frame()

In [237]:
# Выгружаем зелёные зоны (парки, сады, стадионы) города из OpenStreetMap
tags = {"leisure":["park", "garden", "stadium"]}

features_park_samara = ox.features.features_from_place("Samara", tags=tags)

In [ ]:
features_park_samara

In [239]:
type(features_park_samara)

geopandas.geodataframe.GeoDataFrame

In [241]:
# Переводим координаты в геодатафрейме с зелёными зонами города в проекцию EPSG:32637 (UTM Zone 37N)
park_samara_1500 = features_park_samara.to_crs(32637)
# создаём буферную зону в размере 1500 м вокруг каждого геометрического объекта в наборе
park_samara_1500["geometry"] = park_samara_1500["geometry"].buffer(1500)
# переводим полученные координаты в EPSG:4326 (WGS84)
park_samara_1500 = park_samara_1500.to_crs(4326)

In [ ]:
park_samara_1500

In [245]:
# Для дальнейшей работы создаём новый геодатафрейм на основе полученного набора данных
gdf_park_samara_1500 = park_samara_1500["geometry"].to_frame()

In [246]:
type(gdf_park_samara_1500)

geopandas.geodataframe.GeoDataFrame

In [334]:
# В датафрейме с тренировочными данными создаём столбец "green_zone" для указания, попадает ли квартира в зелёную зону
df_train_nodes_copy["green_zone"] = ""

In [ ]:
# Создаём геодатафрейм на основе датасета с тренировочными данными
gdf_train = gpd.GeoDataFrame(df_train_nodes_copy, geometry="geometry")

gdf_train

In [336]:
type(gdf_train)

geopandas.geodataframe.GeoDataFrame

In [ ]:
# С помощью метода "within" определяем, находится ли квартира из тренировочного датасета внутри
# зелёной зоны города (полигона)
count_ = 0 # задаём счётчик для проверки количества проработанных объектов

for i in gdf_train.index:
    if gdf_train.at[i, "green_zone"] == "":
        if gdf_train.at[i, "city"] == "Краснодар":
            is_within = 0 # изначально присваиваем искомому значению резльтат "ноль" (0)
            for poly in gdf_park_krasnodar_1500["geometry"]:
                if gdf_train.at[i, "geometry"].within(poly):
                    is_within = 1 # в случае, если квартира лежит в заданном полигоне, переопределяем переменную на значение "единица" (1)
            # обновляем значение "зелёной зоны" для итерируемой квартиры
            gdf_train.at[i, "green_zone"] = is_within

        elif gdf_train.at[i, "city"] == "Нижний Новгород":
            is_within = 0 # изначально присваиваем искомому значению резльтат "ноль" (0)
            for poly in gdf_park_nizhny_1500["geometry"]:
                if gdf_train.at[i, "geometry"].within(poly):
                    is_within = 1 # в случае, если квартира лежит в заданном полигоне, переопределяем переменную на значение "единица" (1)
            # обновляем значение "зелёной зоны" для итерируемой квартиры
            gdf_train.at[i, "green_zone"] = is_within

        elif gdf_train.at[i, "city"] == "Самара":
            is_within = 0 # изначально присваиваем искомому значению резльтат "ноль" (0)
            for poly in gdf_park_samara_1500["geometry"]:
                if gdf_train.at[i, "geometry"].within(poly):
                    is_within = 1 # в случае, если квартира лежит в заданном полигоне, переопределяем переменную на значение "единица" (1)
            # обновляем значение "зелёной зоны" для итерируемой квартиры
            gdf_train.at[i, "green_zone"] = is_within
            
        count_ += 1
        print(f'Проработано: {count_} объектов')
 

#gdf["green_zone"] = gdf["geometry"].apply(
    #lambda point: gdf_park_krasnodar_1500["geometry"].apply(lambda poly:
        #point.within(poly)).any()
#)

In [338]:
gdf_train

,latitude,longitude,city,area,area_kitchen,rooms_num,has_balcony,price,geometry,geometry_crs,nearest_node,count_stops,green_zone
0,53.2117,50.1669,Самара,33.9,8.0,1,True,2473000,POINT (50.16690 53.21170),POINT (444364.7583481302 5896144.765920336),1376917912,0,1
1,45.0389,39.0985,Краснодар,49.8,12.0,2,True,3733000,POINT (39.09850 45.03890),POINT (507758.0518276638 4987276.430569851),533897701,0,1
2,45.0694,38.9716,Краснодар,45.1,25.1,2,True,3445000,POINT (38.97160 45.06940),POINT (497764.34935612 4990660.299869019),921918400,0,1
3,53.2164,50.2605,Самара,70.1,10.0,3,True,4458000,POINT (50.26050 53.21640),POINT (450620.75961276994 5896598.90707207),10828204246,0,1
4,53.1651,50.3842,Самара,34.0,NaN,1,True,1295000,POINT (50.38420 53.16510),POINT (458831.4735939478 5890813.869220274),597712746,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8037,53.2399,50.2785,Самара,43.0,5.0,2,True,4029000,POINT (50.27850 53.23990),POINT (451849.04119317944 5899200.85693531),1567984170,3,1
8038,45.0739,38.9666,Краснодар,44.9,11.8,1,True,5263000,POINT (38.96660 45.07390),POINT (497370.9552071012 4991160.3494087905),11462814570,0,1
8039,45.1127,39.0288,Краснодар,43.1,12.0,1,False,2017000,POINT (39.02880 45.11270),POINT (502265.426288711 4995470.469169812),7310019142,0,0
8040,45.0513,39.0179,Краснодар,41.1,14.2,1,True,5617000,POINT (39.01790 45.05130),POINT (501409.53431823035 4988649.361517621),295196089,0,1


In [340]:
# Создаём колонку апартаментов (отсутствие в квартире кухни) и заполняем пропуски нулями
gdf_train.loc[(gdf_train["area_kitchen"].isna()) | (gdf_train["area_kitchen"] == 0), "is_apart"] = 1
gdf_train = gdf_train.fillna(0)
gdf_train["is_apart"] = gdf_train["is_apart"].astype(int)

/var/folders/hy/pfmkjt294bl40vdb0nfc_y4c0000gn/T/ipykernel_45384/309369035.py:4: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  gdf_train = gdf_train.fillna(0)


In [ ]:
gdf_train

In [342]:
# Преобразуем колонку балконов в 1 (есть балкон) и 0 (нет балкона)
gdf_train.loc[:, "has_balcony"] = gdf_train.loc[:, "has_balcony"].astype(int)

/var/folders/hy/pfmkjt294bl40vdb0nfc_y4c0000gn/T/ipykernel_45384/2174777982.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[1 1 1 ... 0 1 1]' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  gdf_train.loc[:, "has_balcony"] = gdf_train.loc[:, "has_balcony"].astype(int)


In [ ]:
gdf_train

In [344]:
# Задаём категориальные столбцы и преобразуем их
cat_list = ["city","rooms_num", "has_balcony", "green_zone", "is_apart"]
gdf_train[cat_list] = gdf_train[cat_list].astype("category")

In [ ]:
gdf_train.info()

In [346]:
# Сохраняем полученный датафрейм с тренировочными данными в csv-файл
gdf_train.to_csv("gdf_train_v1.csv")

In [300]:
# Копируем тестовый датасет с квартирами в отдельный датафрейм, чтобы в нём собрать ближайшие узлы
# графа OpenStreetMap для координат квартир в городах
df_test_nodes = df_test.copy()

In [ ]:
# Для этого добавляем столбец "geometry_crs"
df_test_nodes["geometry_crs"] = ""

df_test_nodes

In [ ]:
# Переводим координаты квартир в корректные для городов проекции, определённые ранее 
count_ = 0 # задаём счётчик для проверки количества проработанных объектов

for i in df_test_nodes.index:
    flat_crs = gpd.GeoSeries(df_test_nodes.at[i, "geometry"], crs="EPSG:4326")
    print(f'1 - {flat_crs}')
    if df_test_nodes.at[i, "city"] == "Краснодар":
        df_test_nodes.at[i, "geometry_crs"] = flat_crs.to_crs(edges_1.crs)[0]
        print(f'2 - {flat_crs.to_crs(edges_1.crs)[0]}')
    elif df_test_nodes.at[i, "city"] == "Нижний Новгород":
        df_test_nodes.at[i, "geometry_crs"] = flat_crs.to_crs(edges_2.crs)[0]
    elif df_test_nodes.at[i, "city"] == "Самара":
        df_test_nodes.at[i, "geometry_crs"] = flat_crs.to_crs(edges_3.crs)[0]
        
    count_ += 1
    print(count_)

In [303]:
df_test_nodes

,latitude,longitude,city,area,area_kitchen,rooms_num,has_balcony,geometry,geometry_crs
0,45.0164,39.0467,Краснодар,67.1,NaN,2,True,POINT (39.04670 45.01640),POINT (503679.6241927828 4984773.295640753)
1,45.0729,39.0407,Краснодар,84.1,15.1,3,False,POINT (39.04070 45.07290),POINT (503203.7122498007 4991049.523728226)
2,45.0903,39.0325,Краснодар,34.9,10.0,1,True,POINT (39.03250 45.09030),POINT (502557.4704863871 4992982.178990674)
3,45.1010,38.9272,Краснодар,31.1,8.8,1,True,POINT (38.92720 45.10100),POINT (494272.33584035654 4994172.897198866)
4,45.0751,38.9052,Краснодар,22.9,3.0,1,True,POINT (38.90520 45.07510),POINT (492538.07693616446 4991297.484191589)
...,...,...,...,...,...,...,...,...,...
3438,53.2125,50.2329,Самара,31.0,6.0,1,False,POINT (50.23290 53.21250),POINT (448773.18043347297 5896184.4664658485)
3439,53.2783,50.2692,Самара,56.6,6.2,3,True,POINT (50.26920 53.27830),POINT (451272.05837237067 5903478.930938232)
3440,53.1912,50.0911,Самара,39.0,10.0,0,False,POINT (50.09110 53.19120),POINT (439273.9042863805 5893925.943418219)
3441,53.1917,50.0897,Самара,148.9,13.8,4,True,POINT (50.08970 53.19170),POINT (439181.07624439197 5893982.753111792)


In [ ]:
df_test_nodes["nearest_node"] = ""

df_test_nodes

In [ ]:
# С помощью библиотеки osmnx ищем ближайшие узлы графа OpenStreetMap для координат
# квартир в каждом городе
count_ = 0 # задаём счётчик для проверки количества проработанных объектов

for i in df_test_nodes.index:
    if df_test_nodes.at[i, "city"] == "Краснодар":
        df_test_nodes.at[i, "nearest_node"] = ox.distance.nearest_nodes(G_krasnodar, df_test_nodes.at[i, "geometry_crs"].x, df_test_nodes.at[i, "geometry_crs"].y)
        print(f'1 - {df_test_nodes.at[i, "geometry_crs"]}')
        print(f'2 - {df_test_nodes.at[i, "nearest_node"]}')
    elif df_test_nodes.at[i, "city"] == "Нижний Новгород":
        df_test_nodes.at[i, "nearest_node"] = ox.distance.nearest_nodes(G_nizhny_novgorod, df_test_nodes.at[i, "geometry_crs"].x, df_test_nodes.at[i, "geometry_crs"].y)
        print(f'1 - {df_test_nodes.at[i, "geometry_crs"]}')
        print(f'2 - {df_test_nodes.at[i, "nearest_node"]}')
    elif df_test_nodes.at[i, "city"] == "Самара":
        df_test_nodes.at[i, "nearest_node"] = ox.distance.nearest_nodes(G_samara, df_test_nodes.at[i, "geometry_crs"].x, df_test_nodes.at[i, "geometry_crs"].y)
        print(f'1 - {df_test_nodes.at[i, "geometry_crs"]}')
        print(f'2 - {df_test_nodes.at[i, "nearest_node"]}')
        
    count_ += 1
    print(count_)

In [306]:
df_test_nodes

,latitude,longitude,city,area,area_kitchen,rooms_num,has_balcony,geometry,geometry_crs,nearest_node
0,45.0164,39.0467,Краснодар,67.1,NaN,2,True,POINT (39.04670 45.01640),POINT (503679.6241927828 4984773.295640753),1234702318
1,45.0729,39.0407,Краснодар,84.1,15.1,3,False,POINT (39.04070 45.07290),POINT (503203.7122498007 4991049.523728226),7104549572
2,45.0903,39.0325,Краснодар,34.9,10.0,1,True,POINT (39.03250 45.09030),POINT (502557.4704863871 4992982.178990674),7098473514
3,45.1010,38.9272,Краснодар,31.1,8.8,1,True,POINT (38.92720 45.10100),POINT (494272.33584035654 4994172.897198866),298719791
4,45.0751,38.9052,Краснодар,22.9,3.0,1,True,POINT (38.90520 45.07510),POINT (492538.07693616446 4991297.484191589),11432426761
...,...,...,...,...,...,...,...,...,...,...
3438,53.2125,50.2329,Самара,31.0,6.0,1,False,POINT (50.23290 53.21250),POINT (448773.18043347297 5896184.4664658485),1004368928
3439,53.2783,50.2692,Самара,56.6,6.2,3,True,POINT (50.26920 53.27830),POINT (451272.05837237067 5903478.930938232),596235220
3440,53.1912,50.0911,Самара,39.0,10.0,0,False,POINT (50.09110 53.19120),POINT (439273.9042863805 5893925.943418219),6163938367
3441,53.1917,50.0897,Самара,148.9,13.8,4,True,POINT (50.08970 53.19170),POINT (439181.07624439197 5893982.753111792),1991358690


In [311]:
# Проверяем координаты по ближайшему узлу
G_samara.nodes[1004368928]

{'y': 5896170.701154468,
 'x': 448782.7970286313,
 'street_count': 3,
 'lon': 50.2330462,
 'lat': 53.2123772}

In [312]:
# Сохраняем датафрейм с узлами графа OpenStreetMap в csv-файл
df_test_nodes.to_csv("df_test_nodes.csv")

In [313]:
# Копируем полученный датасет с узлами графа OpenStreetMap, чтобы по этим узлам найти количество
# ближайших остановок, находящихся в пешей доступности от квартиры
df_test_nodes_copy = df_test_nodes.copy()

In [ ]:
df_test_nodes_copy["count_stops"] = ""
df_test_nodes_copy

In [ ]:
# С помощью библиотеки networkx находим остановки, находящиеся от квартиры на расстоянии не более,
# чем в 1 км (1000 м).
# Для этого пройдёмся по датасетам (с остановками в городах и с квартирами)
count_ = 0 # задаём счётчик для проверки количества проработанных объектов

for i in df_test_nodes_copy.index:
    if df_test_nodes_copy.at[i, "count_stops"] == "":
        if df_test_nodes_copy.at[i, "city"] == "Краснодар":
            count_stop = 0 # задаём счётчик для увеличения количества найденных остановок на указанном расстоянии
            for j in stops_krasnodar_crs.index:
                try:
                     '''
                    с помощью библиотеки networkx определяем расстояние между квартирой и городскими остановками
                    по графу OpenStreetMap и сравниваем полученное расстояние с допустимым
                    '''
                    route_length = nx.shortest_path_length(G_krasnodar, df_test_nodes_copy.at[i, "nearest_node"], stops_krasnodar_crs.at[j, "nearest_node"], weight="length")
                    if route_length <= 1000:
                        '''
                        в случае нахождения остановки на допустимом расстоянии
                        увеличиваем количество остановок в счётчике для итерируемой квартиры
                        '''
                        count_stop += 1
                except ValueError:
                    continue
            # обновляем значение количества остановок для итерируемой квартиры
            df_test_nodes_copy.at[i, "count_stops"] = count_stop

        elif df_test_nodes_copy.at[i, "city"] == "Нижний Новгород":
            count_stop = 0
            for j in stops_nizhny_crs.index:
                try:
                     '''
                    с помощью библиотеки networkx определяем расстояние между квартирой и городскими остановками
                    по графу OpenStreetMap и сравниваем полученное расстояние с допустимым
                    '''
                    route_length = nx.shortest_path_length(G_nizhny_novgorod, df_test_nodes_copy.at[i, "nearest_node"], stops_nizhny_crs.at[j, "nearest_node"], weight="length")
                    if route_length <= 1000:
                        '''
                        в случае нахождения остановки на допустимом расстоянии
                        увеличиваем количество остановок в счётчике для итерируемой квартиры
                        '''
                        count_stop += 1
                except ValueError:
                    continue
            # обновляем значение количества остановок для итерируемой квартиры
            df_test_nodes_copy.at[i, "count_stops"] = count_stop

        elif df_test_nodes_copy.at[i, "city"] == "Самара":
            count_stop = 0
            for j in stops_samara_crs.index:
                try:
                     '''
                    с помощью библиотеки networkx определяем расстояние между квартирой и городскими остановками
                    по графу OpenStreetMap и сравниваем полученное расстояние с допустимым
                    '''
                    route_length = nx.shortest_path_length(G_samara, df_test_nodes_copy.at[i, "nearest_node"], stops_samara_crs.at[j, "nearest_node"], weight="length")
                    if route_length <= 1000:
                        '''
                        в случае нахождения остановки на допустимом расстоянии
                        увеличиваем количество остановок в счётчике для итерируемой квартиры
                        '''
                        count_stop += 1
                except ValueError:
                    continue
            # обновляем значение количества остановок для итерируемой квартиры
            df_test_nodes_copy.at[i, "count_stops"] = count_stop
        count_ += 1
        print(f'Проработано: {count_} объектов')
    else:
        continue

In [316]:
# Сохраняем датафрейм с количеством найденных остановок в csv-файл
df_test_nodes_copy.to_csv("df_test_stops.csv")

In [317]:
# В датафрейме с тестовыми данными создаём столбец "green_zone" для указания, попадает ли квартира в зелёную зону
df_test_nodes_copy["green_zone"] = ""

In [ ]:
# Создаём геодатафрейм на основе датасета с тестовыми данными
gdf_test = gpd.GeoDataFrame(df_test_nodes_copy, geometry="geometry")

gdf_test

In [ ]:
# С помощью метода "within" определяем, находится ли квартира из тренировочного датасета внутри
# зелёной зоны города (полигона)
count_ = 0 # задаём счётчик для проверки количества проработанных объектов

for i in gdf_test.index:
    if gdf_test.at[i, "green_zone"] == "":
        if gdf_test.at[i, "city"] == "Краснодар":
            is_within = 0 # изначально присваиваем искомому значению резльтат "ноль" (0)
            for poly in gdf_park_krasnodar_1500["geometry"]:
                if gdf_test.at[i, "geometry"].within(poly):
                    is_within = 1 # в случае, если квартира лежит в заданном полигоне, переопределяем переменную на значение "единица" (1)
            # обновляем значение "зелёной зоны" для итерируемой квартиры
            gdf_test.at[i, "green_zone"] = is_within

        elif gdf_test.at[i, "city"] == "Нижний Новгород":
            is_within = 0 # изначально присваиваем искомому значению резльтат "ноль" (0)
            for poly in gdf_park_nizhny_1500["geometry"]:
                if gdf_test.at[i, "geometry"].within(poly):
                    is_within = 1 # в случае, если квартира лежит в заданном полигоне, переопределяем переменную на значение "единица" (1)
            # обновляем значение "зелёной зоны" для итерируемой квартиры
            gdf_test.at[i, "green_zone"] = is_within

        elif gdf_test.at[i, "city"] == "Самара":
            is_within = 0 # изначально присваиваем искомому значению резльтат "ноль" (0)
            for poly in gdf_park_samara_1500["geometry"]:
                if gdf_test.at[i, "geometry"].within(poly):
                    is_within = 1 # в случае, если квартира лежит в заданном полигоне, переопределяем переменную на значение "единица" (1)
            # обновляем значение "зелёной зоны" для итерируемой квартиры
            gdf_test.at[i, "green_zone"] = is_within
            
        count_ += 1
        print(f'Проработано: {count_} объектов')


#gdf["green_zone"] = gdf["geometry"].apply(
    #lambda point: gdf_park_krasnodar_1500["geometry"].apply(lambda poly:
        #point.within(poly)).any()
#)

In [324]:
gdf_test

,latitude,longitude,city,area,area_kitchen,rooms_num,has_balcony,geometry,geometry_crs,nearest_node,count_stops,green_zone
0,45.0164,39.0467,Краснодар,67.1,NaN,2,True,POINT (39.04670 45.01640),POINT (503679.6241927828 4984773.295640753),1234702318,0,1
1,45.0729,39.0407,Краснодар,84.1,15.1,3,False,POINT (39.04070 45.07290),POINT (503203.7122498007 4991049.523728226),7104549572,0,1
2,45.0903,39.0325,Краснодар,34.9,10.0,1,True,POINT (39.03250 45.09030),POINT (502557.4704863871 4992982.178990674),7098473514,0,1
3,45.1010,38.9272,Краснодар,31.1,8.8,1,True,POINT (38.92720 45.10100),POINT (494272.33584035654 4994172.897198866),298719791,0,1
4,45.0751,38.9052,Краснодар,22.9,3.0,1,True,POINT (38.90520 45.07510),POINT (492538.07693616446 4991297.484191589),11432426761,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
3438,53.2125,50.2329,Самара,31.0,6.0,1,False,POINT (50.23290 53.21250),POINT (448773.18043347297 5896184.4664658485),1004368928,1,1
3439,53.2783,50.2692,Самара,56.6,6.2,3,True,POINT (50.26920 53.27830),POINT (451272.05837237067 5903478.930938232),596235220,0,1
3440,53.1912,50.0911,Самара,39.0,10.0,0,False,POINT (50.09110 53.19120),POINT (439273.9042863805 5893925.943418219),6163938367,1,1
3441,53.1917,50.0897,Самара,148.9,13.8,4,True,POINT (50.08970 53.19170),POINT (439181.07624439197 5893982.753111792),1991358690,1,1


In [332]:
# Создаём колонку апартаментов (отсутствие в квартире кухни) и заполняем пропуски нулями
gdf_test.loc[(gdf_test["area_kitchen"].isna()) | (gdf_test["area_kitchen"] == 0), "is_apart"] = 1
gdf_test = gdf_test.fillna(0)
gdf_test["is_apart"] = gdf_test["is_apart"].astype(int)

gdf_test

,latitude,longitude,city,area,area_kitchen,rooms_num,has_balcony,geometry,geometry_crs,nearest_node,count_stops,green_zone,is_apart
0,45.0164,39.0467,Краснодар,67.1,0.0,2,True,POINT (39.04670 45.01640),POINT (503679.6241927828 4984773.295640753),1234702318,0,1,1
1,45.0729,39.0407,Краснодар,84.1,15.1,3,False,POINT (39.04070 45.07290),POINT (503203.7122498007 4991049.523728226),7104549572,0,1,0
2,45.0903,39.0325,Краснодар,34.9,10.0,1,True,POINT (39.03250 45.09030),POINT (502557.4704863871 4992982.178990674),7098473514,0,1,0
3,45.1010,38.9272,Краснодар,31.1,8.8,1,True,POINT (38.92720 45.10100),POINT (494272.33584035654 4994172.897198866),298719791,0,1,0
4,45.0751,38.9052,Краснодар,22.9,3.0,1,True,POINT (38.90520 45.07510),POINT (492538.07693616446 4991297.484191589),11432426761,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3438,53.2125,50.2329,Самара,31.0,6.0,1,False,POINT (50.23290 53.21250),POINT (448773.18043347297 5896184.4664658485),1004368928,1,1,0
3439,53.2783,50.2692,Самара,56.6,6.2,3,True,POINT (50.26920 53.27830),POINT (451272.05837237067 5903478.930938232),596235220,0,1,0
3440,53.1912,50.0911,Самара,39.0,10.0,0,False,POINT (50.09110 53.19120),POINT (439273.9042863805 5893925.943418219),6163938367,1,1,0
3441,53.1917,50.0897,Самара,148.9,13.8,4,True,POINT (50.08970 53.19170),POINT (439181.07624439197 5893982.753111792),1991358690,1,1,0


In [ ]:
# Преобразуем колонку балконов в 1 (есть балкон) и 0 (нет балкона)
gdf_test.loc[:, "has_balcony"] = gdf_test.loc[:, "has_balcony"].astype(int)

In [348]:
# Задаём категориальные столбцы и преобразуем их
cat_list = ["city","rooms_num", "has_balcony", "green_zone", "is_apart"]
gdf_test[cat_list] = gdf_test[cat_list].astype("category")

In [349]:
gdf_test.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 3443 entries, 0 to 3442
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   latitude      3443 non-null   float64 
 1   longitude     3443 non-null   float64 
 2   city          3443 non-null   category
 3   area          3443 non-null   float64 
 4   area_kitchen  3443 non-null   float64 
 5   rooms_num     3443 non-null   category
 6   has_balcony   3443 non-null   category
 7   geometry      3443 non-null   geometry
 8   geometry_crs  3443 non-null   object  
 9   nearest_node  3443 non-null   int64   
 10  count_stops   3443 non-null   int64   
 11  green_zone    3443 non-null   category
 12  is_apart      3443 non-null   category
dtypes: category(5), float64(4), geometry(1), int64(2), object(1)
memory usage: 388.8+ KB


In [350]:
# Сохраняем полученный датафрейм с тестовыми данными в csv-файл
gdf_test.to_csv("gdf_test_v1.csv")

In [375]:
# Считываем сохранённые данные
gdf_train = pd.read_csv("gdf_train_v1.csv", index_col=[0])
gdf_test = pd.read_csv("gdf_test_v1.csv", index_col=[0])

gdf_train["geometry"] = gdf_train["geometry"].apply(wkt.loads)
gdf_test["geometry"] = gdf_test["geometry"].apply(wkt.loads)

gdf_train = gpd.GeoDataFrame(gdf_train, geometry="geometry", crs=4326)
gdf_test = gpd.GeoDataFrame(gdf_test, geometry="geometry", crs=4326)

In [376]:
# Удаляем ненужные столбцы
drop_list = ["latitude", "longitude", "geometry_crs", "nearest_node"]

gdf_train = gdf_train.drop(drop_list, axis=1)
gdf_test = gdf_test.drop(drop_list, axis=1)

gdf_train.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 8042 entries, 0 to 8041
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   city          8042 non-null   object  
 1   area          8042 non-null   float64 
 2   area_kitchen  8042 non-null   float64 
 3   rooms_num     8042 non-null   int64   
 4   has_balcony   8042 non-null   int64   
 5   price         8042 non-null   int64   
 6   geometry      8042 non-null   geometry
 7   count_stops   8042 non-null   int64   
 8   green_zone    8042 non-null   int64   
 9   is_apart      8042 non-null   int64   
dtypes: float64(2), geometry(1), int64(6), object(1)
memory usage: 691.1+ KB


In [377]:
# Формируем категориальные столбцы заново
cat_data = ["city", "rooms_num", "has_balcony", "green_zone", "is_apart"]
num_data = ["area", "area_kitchen", "count_stops"]

In [378]:
# Преобразуем категориальные столбцы
gdf_train[cat_data] = gdf_train[cat_data].astype("category")
gdf_test[cat_data] = gdf_test[cat_data].astype("category")

In [382]:
gdf_test.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 3443 entries, 0 to 3442
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   city          3443 non-null   category
 1   area          3443 non-null   float64 
 2   area_kitchen  3443 non-null   float64 
 3   rooms_num     3443 non-null   category
 4   has_balcony   3443 non-null   category
 5   geometry      3443 non-null   geometry
 6   count_stops   3443 non-null   int64   
 7   green_zone    3443 non-null   category
 8   is_apart      3443 non-null   category
dtypes: category(5), float64(2), geometry(1), int64(1)
memory usage: 152.1 KB


In [384]:
gdf_train.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 8042 entries, 0 to 8041
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   city          8042 non-null   category
 1   area          8042 non-null   float64 
 2   area_kitchen  8042 non-null   float64 
 3   rooms_num     8042 non-null   category
 4   has_balcony   8042 non-null   category
 5   price         8042 non-null   int64   
 6   geometry      8042 non-null   geometry
 7   count_stops   8042 non-null   int64   
 8   green_zone    8042 non-null   category
 9   is_apart      8042 non-null   category
dtypes: category(5), float64(2), geometry(1), int64(2)
memory usage: 417.1 KB


In [385]:
# Разбиваем данные для обучения
gdf_train, gdf_valid = train_test_split(gdf_train, test_size=0.25, random_state=101)

In [386]:
# Формируем наборы для обучения.
# Разделяем исходные данные (gdf_train) на признаки (features) и целевую переменную (target).
# Признаки создаём путём удаления столбцов "price" и "geometry", а целевой столбец оставляем только "price"
features_train = gdf_train.drop(["price", "geometry"], axis=1)
target_train = gdf_train["price"]

# В валидационном наборе данных также создаём набор признаков без столбца "price" и сохраняем целевую
# переменную "price", чтобы после обучения модели можно было проверить её качество 
features_valid = gdf_valid.drop(["price", "geometry"], axis=1)
target_valid = gdf_valid["price"]

# Удаляем столбец "geometry" для тестовых данных (здесь цена нам неизвестна, мы должны её определить)
features_test = gdf_test.drop(["geometry"], axis=1)

print("Train =", features_train.shape, target_train.shape)
print("Valid =", features_valid.shape, target_valid.shape)
print("Test  =", features_test.shape)

Train = (6031, 8) (6031,)
Valid = (2011, 8) (2011,)
Test  = (3443, 8)


In [388]:
# Задаём пул данных для catboost
train_data = Pool(data=features_train, label=target_train, cat_features=cat_features)
val_data = Pool(data=features_valid, label=target_valid, cat_features=cat_features)

In [389]:
# Задаём параметры модели
param = {
        'eval_metric': 'RMSE',
        'loss_function': 'RMSE',
        'iterations':1000,
        'random_state':101,
        'use_best_model':True,
        'logging_level': 'Silent',
        'learning_rate': 0.01,
        'depth': 12,
    }

In [ ]:
# Обучаем модель
model = CatBoostRegressor(**param)
model.fit(train_data, eval_set=val_data, early_stopping_rounds=100)

In [393]:
# Смотрим результат метрики
rmse = root_mean_squared_error(target_valid, model.predict(features_valid))
rmse

2382060.2090795306

In [394]:
# Смотрим важность признаков от catboost
# Видно, что основной вклад в стоимость кваритиры вносит её площадь
feat_importances = model.get_feature_importance(prettified=True)
feat_importances

,Feature Id,Importances
0,area,59.440608
1,area_kitchen,13.852113
2,city,12.330684
3,rooms_num,7.056841
4,count_stops,2.875937
5,green_zone,2.642063
6,has_balcony,1.795978
7,is_apart,0.005778


In [395]:
# Делаем предсказание
predict = model.predict(features_test)

In [398]:
# Создаём новый датафрейм с предсказанными данными и индексируем его по индексам тестовых данных в "gdf_test"
df_test_sub = pd.Series(predict, index=gdf_test.index)
df_test_sub.name = "price"

In [401]:
# Сохраняем результат с предсказанными ценами на квартиры в csv-файл
df_test_sub.to_csv("sub_v1.csv")

В результате проведения исследования была получена достаточно большая метрика (2382060), которая позволяет сказать, что выбранные геопараметры не влияют на стоимость квартир, и по ним не получится достичь высокоточных результатов в предсказаниях на рынке недвижимости.